# Setup

In [1]:
from manifold_twins import ManifoldTwinsAnalysis
from matplotlib import pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from idrtools import math
from utils import frac_to_mag
import utils

/home/kyle/supernova/snfactory/manifold_twins/analysis/manifold_twins.py:15: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
%matplotlib widget

In [3]:
a = ManifoldTwinsAnalysis()
a.settings['figure_directory'] = './output_standardization/figures/'
a.settings['latex_directory'] = './output_standardization/latex/'
a.run_analysis()

Loading dataset...
    IDR:          BLACKSTON
    Phase range: [-5.0, 5.0] days
    Bin velocity: 1000.0



Estimating the spectra at maximum light...
    Loaded cached stan model
    Using saved stan result
Reading between the lines...
    Loaded cached stan model
    Using saved stan result
Building masks...
    Masking 30/203 targets whose uncertainty power is 
    more than 0.100 of the intrinsic power.
Generating the manifold learning embedding...
Loading other indicators of diversity...


/home/kyle/packages/conda/lib/python3.8/site-packages/numpy/ma/core.py:2794: UserWarning: Warning: converting a masked element to nan.
  _data = np.array(data, dtype=dtype, copy=copy,
/home/kyle/packages/conda/lib/python3.8/site-packages/george/kernels.py:95: RuntimeWarning: divide by zero encountered in log
  log_constant = np.log(float(b)/self.ndim)


Fitting RBTL Twins Manifold GP...
GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.073 ± 0.008
    gp_kernel_amplitude       0.164 ± 0.082
    gp_length_scale           5.541 ± 3.315
    offset                    -0.101 ± 0.124
    covariate_slope_0         -0.122 ± 0.051
    Fit NMAD                  0.083 mag
    Fit std                   0.101 mag
Fitting SALT2 Twins Manifold GP...
GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.085 ± 0.010
    gp_kernel_amplitude       0.380 ± 0.230
    gp_length_scale           8.630 ± 5.327
    offset                    6.956 ± 0.299
    covariate_slope_0         2.810 ± 0.153
    Fit NMAD                  0.105 mag
    Fit std                   0.118 mag
Calculating SALT2 magnitude residuals...
SALT2 magnitude residuals fit: 
    ref_mag: 7.052
    alpha:   0.142
    beta

# Sample selection and attrition

## Look at which SNe~Ia are included or rejected

In [4]:
# Raw RBTL magnitude
plt.figure()

plt.scatter(a.redshifts[a.uncertainty_mask], a.rbtl_mags[a.uncertainty_mask], s=15, c='C3', label='Supernovae rejected by cuts')
plt.scatter(a.redshifts[a.uncertainty_mask & a.redshift_color_mask], a.rbtl_mags[a.uncertainty_mask & a.redshift_color_mask], s=15, c='C0', label='Supernovae passing cuts')

z_range = np.linspace(0.001, 0.09, 100)
pec_vel_disp = a.calculate_peculiar_velocity_uncertainties(z_range)
plt.fill_between(z_range, -pec_vel_disp, pec_vel_disp, alpha=0.2, label='Peculiar velocity dispersion')
plt.axvline(0.02, lw=1, ls='--', c='k', label='Redshift cutoff')

plt.xlim(0.001, 0.09)
plt.ylim(-1, 1.5)
plt.xlabel('Redshift')
plt.ylabel('RBTL measured magnitude')
plt.legend()
a.savefig('rbtl_magnitude.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
# Corrected RBTL magnitude
plt.figure()

plt.scatter(
    a.redshifts[a.uncertainty_mask & a.redshift_color_mask],
    a.residuals_rbtl_gp.residuals[a.uncertainty_mask & a.redshift_color_mask],
    s=15, c='C0', label='Supernovae passing cuts'
)
# plt.scatter(
    # a.redshifts[a.uncertainty_mask & ~a.redshift_color_mask],
    # a.residuals_rbtl_gp.residuals[a.uncertainty_mask & ~a.redshift_color_mask],
    # s=15, c='C3', label='Supernovae rejected by cuts'
# )
plt.xlabel('Redshift')
plt.ylabel('RBTL corrected magnitude')

z_range = np.linspace(0.001, 0.09, 100)
pec_vel_disp = a.calculate_peculiar_velocity_uncertainties(z_range)
plt.fill_between(z_range, -pec_vel_disp, pec_vel_disp, alpha=0.2, label='Peculiar velocity dispersion')
# plt.fill_between(z_range, -2*pec_vel_disp, 2*pec_vel_disp, alpha=0.1, facecolor='C0')
# plt.fill_between(z_range, -3*pec_vel_disp, 3*pec_vel_disp, alpha=0.1, facecolor='C0')

plt.legend()
plt.xlim(0.01, 0.09)
plt.ylim(-0.5, 0.5)
a.savefig('rbtl_corr_magnitude_cut.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
plt.figure()

base_mask = a.uncertainty_mask & a.salt_mask
salt_hr = a.residuals_salt['residuals']

plt.scatter(a.redshifts[base_mask], salt_hr[base_mask], s=15, c='C3', label='Supernovae rejected by cuts')
plt.scatter(a.redshifts[base_mask & a.redshift_color_mask], salt_hr[base_mask & a.redshift_color_mask], s=15, c='C0', label='Supernovae passing cuts')

z_range = np.linspace(0.001, 0.09, 100)
pec_vel_disp = 0.00217 / z_range
plt.fill_between(z_range, -pec_vel_disp, pec_vel_disp, alpha=0.2, label='Peculiar velocity dispersion')
plt.axvline(0.02, lw=1, ls='--', c='k', label='Redshift cutoff')

plt.xlim(0.001, 0.09)
# plt.ylim(-1, 1.5)
plt.ylim(-1, 2.)
plt.xlabel('Redshift')
plt.ylabel('SALT2 measured magnitude')
plt.legend()
plt.tight_layout()
a.savefig('salt_magnitude_redshift.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-7-fd75dc86ab1b>:20: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
  plt.tight_layout()


In [28]:
with a.latex_open('attrition_standardization.tex') as f:
    utils.latex_print(f, "\\textbf{General selection requirements} & \\\\")
    utils.latex_print(f, "All SNe~Ia with at least 5 SNfactory spectra          & %d \\\\" % a.attrition_enough_spectra)
    utils.latex_print(f, "Included in Twins manifold                            & %d \\\\" % np.sum(a.uncertainty_mask))
    utils.latex_print(f, "\hline")
    
    utils.latex_print(f, "\\textbf{Standardization of near-maximum spectra} & \\\\")
    utils.latex_print(f, "Host galaxy redshift available                        & %d \\\\" % np.sum(a.uncertainty_mask & (a.redshift_errs < 0.004)))
    utils.latex_print(f, "Host galaxy redshift above 0.02                       & %d \\\\" % np.sum(a.uncertainty_mask & (a.redshift_errs < 0.004) & (a.redshifts > 0.02)))
    utils.latex_print(f, "Measured $A_V$ < 0.5 mag                              & %d \\\\" % np.sum(a.uncertainty_mask & a.redshift_color_mask))
    utils.latex_print(f, "Blinded training subsample                            & %d \\\\" % np.sum(a.uncertainty_mask & a.redshift_color_mask & a.train_mask))
    utils.latex_print(f, "Validation subsample                                  & %d \\\\" % np.sum(a.uncertainty_mask & a.redshift_color_mask & ~a.train_mask))
    utils.latex_print(f, "\hline")
    
    utils.latex_print(f, "\\textbf{Comparisons to SALT2 standardization} & \\\\")
    utils.latex_print(f, r"\textbf{(Section~\ref{sec:salt2_standardization})} & \\")
    utils.latex_print(f, "SNfactory SALT2 selection requirements                & %d \\\\" % np.sum(a.salt_mask))
    utils.latex_print(f, "Passes host galaxy redshift and color requirements    & %d \\\\" % np.sum(a.salt_mask & a.redshift_color_mask))
    utils.latex_print(f, "Included in the Twins manifold                        & %d \\\\" % np.sum(a.salt_mask & a.uncertainty_mask & a.redshift_color_mask))
    utils.latex_print(f, "Blinded training subsample                            & %d \\\\" % np.sum(a.salt_mask & a.uncertainty_mask & a.redshift_color_mask & a.train_mask))
    utils.latex_print(f, "Validation subsample                                  & %d \\\\" % np.sum(a.salt_mask & a.uncertainty_mask & a.redshift_color_mask & ~a.train_mask))

\textbf{General selection requirements} & \\
All SNe~Ia with at least 5 SNfactory spectra          & 280 \\
Included in Twins manifold                            & 173 \\
\hline
\textbf{Standardization of near-maximum spectra} & \\
Host galaxy redshift available                        & 168 \\
Host galaxy redshift above 0.02                       & 144 \\
Measured $A_V$ < 0.5 mag                              & 134 \\
Blinded training subsample                            & 72 \\
Validation subsample                                  & 62 \\
\hline
\textbf{Comparisons to SALT2 standardization} & \\
\textbf{(Section~\ref{sec:salt2_standardization})} & \\
SNfactory SALT2 selection requirements                & 183 \\
Passes host galaxy redshift and color requirements    & 151 \\
Included in the Twins manifold                        & 127 \\
Blinded training subsample                            & 66 \\
Validation subsample                                  & 61 \\


# Standardization

## Raw magnitudes

In [4]:
print("Raw RBTL mag std:  %.3f mag" % np.std(a.rbtl_mags[a.uncertainty_mask & a.redshift_color_mask]))
print("Raw RBTL mag NMAD: %.3f mag" % math.nmad(a.rbtl_mags[a.uncertainty_mask & a.redshift_color_mask]))

Raw RBTL mag std:  0.130 mag
Raw RBTL mag NMAD: 0.108 mag


In [5]:
a.scatter_combined(a.rbtl_mags, a.uncertainty_mask & a.redshift_color_mask, vmin=-0.3, vmax=0.3, label='Magnitude residuals', invert_colorbar=True)
a.savefig('embedding_residual_magnitudes.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## GP standardization

In [6]:
a.residuals_rbtl_gp.plot()
a.savefig('rbtl_gp_predictions.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Standardization summary

In [24]:
with a.latex_open('standardization_summary.tex') as f:
    # Raw RBTL magnitudes
    raw_rbtl_mags = a.rbtl_mags[a.uncertainty_mask & a.redshift_color_mask]
    utils.latex_std(f, 'rawrbtlmagstd', raw_rbtl_mags)
    utils.latex_nmad(f, 'rawrbtlmagnmad', raw_rbtl_mags)

    # RBTL GP
    rbtl_gp = a.residuals_rbtl_gp
    rbtl_gp_mags = rbtl_gp.residuals[rbtl_gp.mask]
    utils.latex_print(f, '')
    utils.latex_std(f, 'rbtlgprms', rbtl_gp_mags)
    utils.latex_nmad(f, 'rbtlgpnmad', rbtl_gp_mags)
    fiducial_rv = a.settings['rbtl_fiducial_rv']
    utils.latex_command(f, 'rbtlgpintdisp', '%.3f $\\pm$ %.3f', (rbtl_gp.parameters[0], rbtl_gp.parameter_uncertainties[0]))
    utils.latex_command(f, 'rbtlgpkernelamp', '%.3f $\\pm$ %.3f', (np.abs(rbtl_gp.parameters[1]), rbtl_gp.parameter_uncertainties[1]))
    utils.latex_command(f, 'rbtlgpkernellengthscale', '%.2f $\\pm$ %.2f', (rbtl_gp.parameters[2], rbtl_gp.parameter_uncertainties[2]))
    utils.latex_command(f, 'rbtlgpoffset', '%.3f $\\pm$ %.3f', (rbtl_gp.parameters[3], rbtl_gp.parameter_uncertainties[3]))
    utils.latex_command(f, 'rbtlgprv', '%.2f $\\pm$ %.2f', (fiducial_rv * (1 + rbtl_gp.parameters[4]), fiducial_rv * rbtl_gp.parameter_uncertainties[4]))

    # SALT GP
    salt_gp = a.residuals_salt_gp
    salt_gp_mags = salt_gp.residuals[salt_gp.mask]
    utils.latex_print(f, '')
    utils.latex_std(f, 'saltgprms', salt_gp_mags)
    utils.latex_nmad(f, 'saltgpnmad', salt_gp_mags)
    utils.latex_command(f, 'saltgpintdisp', '%.3f $\\pm$ %.3f', (salt_gp.parameters[0], salt_gp.parameter_uncertainties[0]))
    utils.latex_command(f, 'saltgpkernelamp', '%.3f $\\pm$ %.3f', (np.abs(salt_gp.parameters[1]), salt_gp.parameter_uncertainties[1]))
    utils.latex_command(f, 'saltgpkernellengthscale', '%.2f $\\pm$ %.2f', (salt_gp.parameters[2], salt_gp.parameter_uncertainties[2]))
    utils.latex_command(f, 'saltgpoffset', '%.3f $\\pm$ %.3f', (salt_gp.parameters[3], salt_gp.parameter_uncertainties[3]))
    utils.latex_command(f, 'saltgpcolor', '%.2f $\\pm$ %.2f', (salt_gp.parameters[4], salt_gp.parameter_uncertainties[4]))
    rbtl_gp_mags_salt_comp = rbtl_gp.residuals[salt_gp.mask]
    utils.latex_std(f, 'rbtlgpsaltcomprms', rbtl_gp_mags_salt_comp)
    utils.latex_nmad(f, 'rbtlgpsaltcompnmad', rbtl_gp_mags_salt_comp)

    # Peculiar velocity contribution
    utils.latex_print(f, "")
    utils.latex_command(f, 'pecvelcontribution', '%.3f', np.sqrt(np.mean(a.calculate_peculiar_velocity_uncertainties(a.redshifts)[rbtl_gp.mask]**2)))

\newcommand{\rawrbtlmagstd}{0.131 $\pm$ 0.010}
\newcommand{\rawrbtlmagnmad}{0.108 $\pm$ 0.013}

\newcommand{\rbtlgprms}{0.101 $\pm$ 0.007}
\newcommand{\rbtlgpnmad}{0.083 $\pm$ 0.010}
\newcommand{\rbtlgpintdisp}{0.073 $\pm$ 0.008}
\newcommand{\rbtlgpkernelamp}{0.164 $\pm$ 0.082}
\newcommand{\rbtlgpkernellengthscale}{5.54 $\pm$ 3.32}
\newcommand{\rbtlgpoffset}{-0.101 $\pm$ 0.124}
\newcommand{\rbtlgprv}{2.46 $\pm$ 0.14}

\newcommand{\saltgprms}{0.118 $\pm$ 0.008}
\newcommand{\saltgpnmad}{0.105 $\pm$ 0.013}
\newcommand{\saltgpintdisp}{0.085 $\pm$ 0.010}
\newcommand{\saltgpkernelamp}{0.380 $\pm$ 0.230}
\newcommand{\saltgpkernellengthscale}{8.63 $\pm$ 5.33}
\newcommand{\saltgpoffset}{6.956 $\pm$ 0.299}
\newcommand{\saltgpcolor}{2.81 $\pm$ 0.15}
\newcommand{\rbtlgpsaltcomprms}{0.100 $\pm$ 0.008}
\newcommand{\rbtlgpsaltcompnmad}{0.083 $\pm$ 0.011}

\newcommand{\pecvelcontribution}{0.055}


# SALT2 comparison

## SALT2 fit

## SALT2 magnitudes vs components

In [5]:
a.scatter_combined(a.residuals_salt['residuals'], mask=a.salt_mask & a.redshift_color_mask & a.uncertainty_mask, vmin=-0.3, vmax=0.3, label='SALT2-corrected magnitude residuals', invert_colorbar=True)
a.savefig('salt2_hr_embedding.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
plt.figure()

use_x = a.embedding[:, 0]

mask = a.salt_mask & a.redshift_color_mask & a.uncertainty_mask
residuals = a.residuals_salt['residuals']
residual_uncertainties = a.residuals_salt['residual_uncertainties']

plt.errorbar(use_x[mask], residuals[mask], residual_uncertainties[mask], label='Individual SNe Ia', fmt='.', alpha=0.25, c='k')

mask_91t = a.indicators['peculiar_type'] == '91T-like'
plt.scatter(use_x[mask_91t & mask], residuals[mask_91t & mask], label='91T-like SNe Ia', s=15, marker='s', zorder=10)

from scipy.stats import binned_statistic
binned_mean, bin_edges, binnumber = binned_statistic(use_x[mask], residuals[mask], 'mean', bins=10)
binned_var_num, bin_edges, binnumber = binned_statistic(use_x[mask], residual_uncertainties[mask]**2, 'sum', bins=10)
binned_var_denom, bin_edges, binnumber = binned_statistic(use_x[mask], residual_uncertainties[mask]**2, 'count', bins=10)
binned_yerr = np.sqrt(binned_var_num / binned_var_denom**2)

bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2.
plt.step(bin_edges, np.hstack([binned_mean, binned_mean[-1]]), where='post', c='C3', lw=2, label='Binned mean')
plt.errorbar(bin_centers, binned_mean, yerr=binned_yerr, fmt='none', c='C3', lw=2)

plt.legend()

plt.gca().invert_yaxis()

plt.axhline(0, c='k', ls='--', lw=1)

plt.xlabel('Twins Manifold Component 1')
plt.ylabel('SALT2 Hubble residuals')

plt.tight_layout()
a.savefig('salt2_hr_component_1.pdf')

plt.figure()
plt.hist(residuals[(use_x < 3) & mask], 10, (-0.6, 0.4), alpha=0.3, color='C0', label='Component 1 < 2', density=True)
plt.hist(residuals[(use_x < 3) & mask], 10, (-0.6, 0.4), histtype='step', lw=2, color='C0', density=True)
plt.hist(residuals[(use_x > 3) & mask], 10, (-0.6, 0.4), alpha=0.3, color='C1', label='Component 1 > 2', density=True)
plt.hist(residuals[(use_x > 3) & mask], 10, (-0.6, 0.4), histtype='step', lw=2, color='C1', density=True)

plt.gca().invert_xaxis()

plt.xlabel('SALT2 residual magnitude')
plt.ylabel('Normalized counts')
plt.legend()

plt.tight_layout()
a.savefig('salt2_hr_hist.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-60-c09598adb027>:33: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
  plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-60-c09598adb027>:48: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
  plt.tight_layout()


## SALT2 + Isomap standardization

In [7]:
residuals_salt_gp = a.fit_gp_magnitude_residuals(kind='salt_raw', verbosity=2)
residuals_salt_gp.plot(vmin=-0.5, vmax=0.5)
a.savefig('salt_gp_standardization.pdf')

/home/kyle/packages/conda/lib/python3.8/site-packages/george/kernels.py:95: RuntimeWarning: divide by zero encountered in log
  log_constant = np.log(float(b)/self.ndim)


GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.085 ± 0.010
    gp_kernel_amplitude       0.380 ± 0.230
    gp_length_scale           8.630 ± 5.327
    offset                    6.956 ± 0.299
    covariate_slope_0         2.810 ± 0.153
    Fit NMAD                  0.105 mag
    Fit std                   0.117 mag


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
plt.figure()
m = a.salt_mask & a.uncertainty_mask & a.redshift_color_mask
gp_residuals = residuals_salt_gp.residuals
salt_residuals = a.residuals_salt['residuals']
plt.scatter(a.salt_x1[m], (gp_residuals - salt_residuals)[m], c=a.embedding[m, 0], cmap=plt.cm.coolwarm, s=a.settings['scatter_plot_marker_size'], edgecolors='gray')
plt.gca().invert_yaxis()
plt.colorbar(label='Value of Twins Manifold Component 1', aspect=30)
plt.xlabel('SALT2 $x_1$')
# plt.ylabel('Difference between SALT2 + twins manifold\n and SALT2 standardization (mag)')
# plt.ylabel('Difference between SALT2 + Twins Manifold\n and SALT2 + $x_1$ standardization (mag)')
plt.ylabel('Magnitudes')
a.savefig('salt_x1_manifold_difference.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Host galaxy correlations

## Simultaneous fits

In [7]:
# RBTL with SALT and Rigault cuts
print("Both:")
a.fit_gp_magnitude_residuals(
    kind='rbtl', mask=a.host_mask & a.salt_mask & a.peculiar_mask,
    additional_covariates=[a.indicators['host_p(highgmass)'], a.indicators['host_p(prompt)']],
)
print("\nGlobal mass:")
a.fit_gp_magnitude_residuals(
    kind='rbtl', mask=a.host_mask & a.salt_mask & a.peculiar_mask,
    additional_covariates=[a.indicators['host_p(highgmass)']]
)
print("\nLSSFR:")
gp = a.fit_gp_magnitude_residuals(
    kind='rbtl', mask=a.host_mask & a.salt_mask & a.peculiar_mask,
    additional_covariates=[a.indicators['host_p(prompt)']]
)

Both:


/home/kyle/packages/conda/lib/python3.8/site-packages/george/kernels.py:95: RuntimeWarning: divide by zero encountered in log
  log_constant = np.log(float(b)/self.ndim)


GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.067 ± 0.008
    gp_kernel_amplitude       0.097 ± 0.049
    gp_length_scale           4.610 ± 3.202
    offset                    -0.076 ± 0.076
    covariate_slope_0         -0.122 ± 0.056
    covariate_slope_1         -0.013 ± 0.023
    covariate_slope_2         0.058 ± 0.025
    Fit NMAD                  0.066 mag
    Fit std                   0.097 mag

Global mass:
GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.068 ± 0.010
    gp_kernel_amplitude       0.087 ± 0.036
    gp_length_scale           3.423 ± 2.393
    offset                    -0.025 ± 0.060
    covariate_slope_0         -0.123 ± 0.058
    covariate_slope_1         -0.040 ± 0.020
    Fit NMAD                  0.074 mag
    Fit std                   0.099 mag

LSSFR:
GP magnitude residuals fit:
  

In [8]:
# RBTL No SALT mask
print("Both:")
a.fit_gp_magnitude_residuals(
    kind='rbtl', mask=a.host_mask,
    additional_covariates=[a.indicators['host_p(highgmass)'], a.indicators['host_p(prompt)']],
)
print("\nGlobal mass:")
a.fit_gp_magnitude_residuals(
    kind='rbtl', mask=a.host_mask,
    additional_covariates=[a.indicators['host_p(highgmass)']]
)
print("\nLSSFR:")
gp = a.fit_gp_magnitude_residuals(
    kind='rbtl', mask=a.host_mask,
    additional_covariates=[a.indicators['host_p(prompt)']]
)

Both:
GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.072 ± 0.008
    gp_kernel_amplitude       0.221 ± 0.148
    gp_length_scale           9.399 ± 6.412
    offset                    -0.184 ± 0.199
    covariate_slope_0         -0.124 ± 0.058
    covariate_slope_1         -0.008 ± 0.023
    covariate_slope_2         0.053 ± 0.024
    Fit NMAD                  0.076 mag
    Fit std                   0.100 mag

Global mass:
GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.073 ± 0.008
    gp_kernel_amplitude       0.191 ± 0.118
    gp_length_scale           7.582 ± 5.090
    offset                    -0.121 ± 0.163
    covariate_slope_0         -0.122 ± 0.059
    covariate_slope_1         -0.032 ± 0.020
    Fit NMAD                  0.077 mag
    Fit std                   0.102 mag

LSSFR:
GP magnitude residuals f

In [9]:
# SALT + manifold with Rigault cuts
print("Both:")
a.fit_gp_magnitude_residuals(
    kind='salt_raw', mask=a.host_mask & a.salt_mask & a.peculiar_mask,
    additional_covariates=[a.indicators['host_p(highgmass)'], a.indicators['host_p(prompt)']],
)
print("\nGlobal mass:")
a.fit_gp_magnitude_residuals(
    kind='salt_raw', mask=a.host_mask & a.salt_mask & a.peculiar_mask,
    additional_covariates=[a.indicators['host_p(highgmass)']]
)
print("\nLSSFR:")
gp = a.fit_gp_magnitude_residuals(
    kind='salt_raw', mask=a.host_mask & a.salt_mask & a.peculiar_mask,
    additional_covariates=[a.indicators['host_p(prompt)']]
)

Both:
GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.081 ± 0.011
    gp_kernel_amplitude       0.360 ± 0.247
    gp_length_scale           9.214 ± 6.406
    offset                    7.029 ± 0.292
    covariate_slope_0         2.794 ± 0.166
    covariate_slope_1         -0.014 ± 0.029
    covariate_slope_2         0.045 ± 0.031
    Fit NMAD                  0.117 mag
    Fit std                   0.116 mag

Global mass:
GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.082 ± 0.011
    gp_kernel_amplitude       0.329 ± 0.220
    gp_length_scale           8.353 ± 5.918
    offset                    7.057 ± 0.261
    covariate_slope_0         2.793 ± 0.167
    covariate_slope_1         -0.036 ± 0.025
    Fit NMAD                  0.121 mag
    Fit std                   0.117 mag

LSSFR:
GP magnitude residuals fit:


In [10]:
ref, bootstrap = a.bootstrap_salt_magnitude_residuals(
    mask=a.host_mask & a.salt_mask & a.peculiar_mask,
    num_samples=100,
    additional_covariates=[a.indicators['host_p(prompt)'], a.indicators['host_p(highgmass)']]
)

In [11]:
var = 'covariate_amplitude_0'
print(f'{ref[var]:.3f}, {np.mean(bootstrap[var]):.3f}, {np.std(bootstrap[var]):.3f}')
var = 'covariate_amplitude_1'
print(f'{ref[var]:.3f}, {np.mean(bootstrap[var]):.3f}, {np.std(bootstrap[var]):.3f}')

0.103, 0.102, 0.033
-0.060, -0.063, 0.027


In [12]:
a.fit_salt_magnitude_residuals(
    mask=a.host_mask & a.salt_mask & a.peculiar_mask,
    additional_covariates=[a.indicators['host_p(highgmass)'], a.indicators['host_p(prompt)']]
)

a.fit_salt_magnitude_residuals(
    mask=a.host_mask & a.salt_mask & a.peculiar_mask,
    additional_covariates=[a.indicators['host_p(highgmass)']]
)

a.fit_salt_magnitude_residuals(
    mask=a.host_mask & a.salt_mask & a.peculiar_mask,
    additional_covariates=[a.indicators['host_p(prompt)']]
)

print("Done!")

SALT2 magnitude residuals fit: 
    ref_mag: 7.049
    alpha:   0.165
    beta:    2.755
    σ_int:   0.081
    RMS:     0.113
    NMAD:    0.116
    WRMS:    0.112
    amp[0]:  -0.060
    amp[1]:  0.103
SALT2 magnitude residuals fit: 
    ref_mag: 7.127
    alpha:   0.152
    beta:    2.753
    σ_int:   0.089
    RMS:     0.118
    NMAD:    0.106
    WRMS:    0.117
    amp[0]:  -0.108
SALT2 magnitude residuals fit: 
    ref_mag: 6.998
    alpha:   0.162
    beta:    2.692
    σ_int:   0.085
    RMS:     0.115
    NMAD:    0.124
    WRMS:    0.114
    amp[0]:  0.138
Done!


## Residuals fits

In [13]:
import importlib
import hoststep
importlib.reload(hoststep)
from hoststep import fit_step, plot_step

In [14]:
mask = a.uncertainty_mask & a.redshift_color_mask & a.host_mask & a.salt_mask & a.peculiar_mask

salt_fit = a.fit_salt_magnitude_residuals(
    mask=mask,
    # additional_covariates=[a.indicators['host_p(prompt)']]
)

print("")
gp_fit = a.fit_gp_magnitude_residuals(
    kind='rbtl', mask=mask,
    # additional_covariates=[a.indicators['host_p(prompt)']]
)

print("")
salt_gp_fit = a.fit_gp_magnitude_residuals(
    kind='salt_raw', mask=mask,
    # additional_covariates=[a.indicators['host_p(prompt)']]
)

print("\nSALT2:")
print("  Global mass:")
fit_step(a.indicators['host_p(highgmass)'], salt_fit['residuals'], salt_fit['raw_residual_uncertainties'], mask)
print("  LSSFR:")
fit_step(a.indicators['host_p(prompt)'], salt_fit['residuals'], salt_fit['raw_residual_uncertainties'], mask)
print("\nRBTL GP:")
print("  Global mass:")
fit_step(a.indicators['host_p(highgmass)'], gp_fit.residuals, gp_fit.raw_residual_uncertainties, mask)
print("  LSSFR:")
fit_step(a.indicators['host_p(prompt)'], gp_fit.residuals, gp_fit.raw_residual_uncertainties, mask)
print("\nSALT2 GP:")
print("  Global mass:")
fit_step(a.indicators['host_p(highgmass)'], salt_gp_fit.residuals, salt_gp_fit.raw_residual_uncertainties, mask)
print("  LSSFR:")
fit_step(a.indicators['host_p(prompt)'], salt_gp_fit.residuals, salt_gp_fit.raw_residual_uncertainties, mask)

print("\nBase SALT2:")
print("  Global mass:")
fit_step(a.indicators['host_p(highgmass)'], a.residuals_salt['residuals'], a.residuals_salt['raw_residual_uncertainties'], mask)
print("  LSSFR:")
fit_step(a.indicators['host_p(prompt)'], a.residuals_salt['residuals'], a.residuals_salt['raw_residual_uncertainties'], mask)
print("\nBase RBTL GP:")
print("  Global mass:")
fit_step(a.indicators['host_p(highgmass)'], a.residuals_rbtl_gp.residuals, a.residuals_rbtl_gp.raw_residual_uncertainties, mask)
print("  LSSFR:")
step = fit_step(a.indicators['host_p(prompt)'], a.residuals_rbtl_gp.residuals, a.residuals_rbtl_gp.raw_residual_uncertainties, mask)

print("Done!")

SALT2 magnitude residuals fit: 
    ref_mag: 7.067
    alpha:   0.137
    beta:    2.730
    σ_int:   0.091
    RMS:     0.118
    NMAD:    0.096
    WRMS:    0.118

GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.068 ± 0.010
    gp_kernel_amplitude       0.085 ± 0.030
    gp_length_scale           2.813 ± 1.749
    offset                    -0.047 ± 0.053
    covariate_slope_0         -0.140 ± 0.058
    Fit NMAD                  0.087 mag
    Fit std                   0.100 mag

GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.083 ± 0.011
    gp_kernel_amplitude       0.304 ± 0.196
    gp_length_scale           7.597 ± 5.351
    offset                    7.021 ± 0.236
    covariate_slope_0         2.749 ± 0.167
    Fit NMAD                  0.119 mag
    Fit std                   0.118 mag

SALT2:
  Global mass

In [15]:
# residuals = salt_fit['residuals']
residuals = gp_fit.residuals
chauvenet_mask = np.sum(mask) * np.exp(-(residuals[mask] / np.std(residuals[mask]))**2 / 2) / np.sqrt(2 * np.pi) < 0.5
print("Names:        ", [i.name for i in a.targets[mask][chauvenet_mask]])
print("P(prompt):    ", ['%.3f' % i for i in a.indicators['host_p(prompt)'][mask][chauvenet_mask]])
print("P(highgmass): ", ['%.3f' % i for i in a.indicators['host_p(highgmass)'][mask][chauvenet_mask]])
print("SALT2 + x1:   ", salt_fit['residuals'][mask][chauvenet_mask])
print("GP + manifold:", gp_fit.residuals[mask][chauvenet_mask])

mm = np.zeros(len(a.targets), dtype=bool)
mm[mask] = chauvenet_mask

Names:         ['PTF10hmv', 'SNF20080612-003']
P(prompt):     ['0.965', '0.001']
P(highgmass):  ['0.000', '0.196']
SALT2 + x1:    [ 0.39292337 -0.23631362]
GP + manifold: [ 0.31895559 -0.31883272]


In [16]:
ax1, ax2 = plot_step(
    'host_lssfr',
    gp_fit.residuals,
    gp_fit.target_value_uncertainties,
    a.indicators,
    mask,
)

    Step: +0.046 ± 0.017 mag, σ1: 0.080 ± 0.012 mag, σ2: 0.067 ± 0.010 mag


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
ax1, ax2 = plot_step(
    'host_gmass',
    gp_fit.residuals,
    gp_fit.target_value_uncertainties,
    a.indicators,
    mask
)

    Step: -0.031 ± 0.018 mag, σ1: 0.086 ± 0.014 mag, σ2: 0.067 ± 0.010 mag


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kyle/packages/conda/lib/python3.8/site-packages/numpy/core/_asarray.py:85: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order)


## Step difference

In [18]:
def bootstrap_step_difference(side_probabilities, residuals_1, uncertainties_1, residuals_2, uncertainties_2,
                              mask, num_resamples=100):
    mask_indices = np.where(mask)[0]
    
    ref_step_result_1 = fit_step(side_probabilities, residuals_1, uncertainties_1, mask, verbosity=0)
    ref_step_result_2 = fit_step(side_probabilities, residuals_2, uncertainties_2, mask, verbosity=0)

    step_diffs = []
    for bootstrap_iter in tqdm(range(num_resamples)):
        bootstrap_idx = np.random.choice(len(mask_indices), len(mask_indices))
        bootstrap_mask = mask_indices[bootstrap_idx]
        
        step_result_1 = fit_step(side_probabilities, residuals_1, uncertainties_1, bootstrap_mask, verbosity=0, calculate_covariance=False)
        step_result_2 = fit_step(side_probabilities, residuals_2, uncertainties_2, bootstrap_mask, verbosity=0, calculate_covariance=False)
        
        step_diffs.append(step_result_1['step_size'] - step_result_2['step_size'])
        
    plt.figure()
    plt.hist(step_diffs, 30)

    return np.mean(step_diffs), np.std(step_diffs)

In [19]:
bootstrap_step_difference(
    a.indicators['host_p(highgmass)'],
    gp_fit.residuals,
    gp_fit.target_value_uncertainties,
    salt_fit['residuals'],
    salt_fit['raw_residual_uncertainties'],
    mask,
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.05176040703098535, 0.014633940261161075)

## Summary plot

In [27]:
mask = a.redshift_color_mask & a.uncertainty_mask & a.host_mask & a.salt_mask & a.peculiar_mask
# mask = a.uncertainty_mask & a.host_mask & a.salt_mask & a.peculiar_mask

step_vars = [
    ('host_gmass', 'host_p(highgmass)', 'Global Mass', -1),
    ('host_lssfr', 'host_p(prompt)', 'Local SSFR', +1),
]

mag_vars = [
    (False, 'salt', 'SALT2 + $x_1$'),
    (True, 'salt_raw', 'SALT2 + Manifold'),
    (True, 'rbtl', 'RBTL + Manifold'),
]

host_results = {}

for var_name, side_probability, var_label, var_sign in step_vars:
    steps = {}
    for use_gp, mag_kind, mag_label in mag_vars:
        if use_gp:
            result = a.fit_gp_magnitude_residuals(
                kind=mag_kind,
                mask=mask,
                additional_covariates=[a.indicators[side_probability]],
            )
            step = result.parameters[-1]
            step_uncertainty = result.parameter_uncertainties[-1]
        else:
            ref, bootstrap = a.bootstrap_salt_magnitude_residuals(
                mask=mask,
                num_samples=100,
                additional_covariates=[a.indicators[side_probability]],
            )
            step = ref['covariate_amplitude_0']
            step_uncertainty = np.std(bootstrap['covariate_amplitude_0'])
        
        steps[mag_label] = (step * var_sign, step_uncertainty)

    host_results[var_label + '\nSimultaneous Fit'] = steps
    

for var_name, side_probability, var_label, var_sign in step_vars:
    steps = {}
    for use_gp, mag_kind, mag_label in mag_vars:
        if use_gp:
            gp_fit = a.fit_gp_magnitude_residuals(kind=mag_kind, mask=mask)
            residuals = gp_fit.residuals
            residual_uncertainties = gp_fit.raw_residual_uncertainties
        else:
            salt_fit = a.fit_salt_magnitude_residuals(mask=mask)
            residuals = salt_fit['residuals']
            residual_uncertainties = salt_fit['raw_residual_uncertainties']

        step_result = fit_step(
            a.indicators[side_probability],
            residuals,
            residual_uncertainties,
            mask
        )

        steps[mag_label] = (step_result['step_size'] * var_sign, step_result['step_size_uncertainty'])

    host_results[var_label + '\nAfter Correction'] = steps


GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.082 ± 0.011
    gp_kernel_amplitude       0.329 ± 0.220
    gp_length_scale           8.353 ± 5.918
    offset                    7.057 ± 0.261
    covariate_slope_0         2.793 ± 0.167
    covariate_slope_1         -0.036 ± 0.025
    Fit NMAD                  0.121 mag
    Fit std                   0.117 mag
GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.068 ± 0.010
    gp_kernel_amplitude       0.087 ± 0.036
    gp_length_scale           3.423 ± 2.393
    offset                    -0.025 ± 0.060
    covariate_slope_0         -0.123 ± 0.058
    covariate_slope_1         -0.040 ± 0.020
    Fit NMAD                  0.074 mag
    Fit std                   0.099 mag



GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.081 ± 0.011
    gp_kernel_amplitude       0.359 ± 0.245
    gp_length_scale           9.191 ± 6.381
    offset                    7.012 ± 0.289
    covariate_slope_0         2.781 ± 0.164
    covariate_slope_1         0.053 ± 0.027
    Fit NMAD                  0.124 mag
    Fit std                   0.116 mag
GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.068 ± 0.008
    gp_kernel_amplitude       0.097 ± 0.048
    gp_length_scale           4.543 ± 3.120
    offset                    -0.089 ± 0.073
    covariate_slope_0         -0.126 ± 0.056
    covariate_slope_1         0.066 ± 0.022
    Fit NMAD                  0.077 mag
    Fit std                   0.097 mag
SALT2 magnitude residuals fit: 
    ref_mag: 7.067
    alpha:   0.137
    beta:    2.730
    σ_int:

/home/kyle/packages/conda/lib/python3.8/site-packages/george/kernels.py:95: RuntimeWarning: divide by zero encountered in log
  log_constant = np.log(float(b)/self.ndim)


GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.083 ± 0.011
    gp_kernel_amplitude       0.304 ± 0.196
    gp_length_scale           7.597 ± 5.351
    offset                    7.021 ± 0.236
    covariate_slope_0         2.749 ± 0.167
    Fit NMAD                  0.119 mag
    Fit std                   0.118 mag
    Step: -0.030 ± 0.023 mag, σ1: 0.093 ± 0.017 mag, σ2: 0.071 ± 0.015 mag
GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.068 ± 0.010
    gp_kernel_amplitude       0.085 ± 0.030
    gp_length_scale           2.813 ± 1.749
    offset                    -0.047 ± 0.053
    covariate_slope_0         -0.140 ± 0.058
    Fit NMAD                  0.087 mag
    Fit std                   0.100 mag
    Step: -0.032 ± 0.018 mag, σ1: 0.078 ± 0.016 mag, σ2: 0.057 ± 0.012 mag
SALT2 magnitude residuals fit: 
    re

In [28]:
plt.figure(figsize=(6, 4))
labels = []
for prop_idx, (prop_label, prop_values) in enumerate(host_results.items()):
    for mag_idx, (mag_label, mag_values) in enumerate(prop_values.items()):
        step_value, step_err = mag_values
        
        marker = 'oooo'[mag_idx]
        color = 'C%d' % (mag_idx)
        if prop_idx == 0:
            label = mag_label
        else:
            label = None
            
            
        gap = 0.1
        xpos = prop_idx - 0.5*gap + gap * mag_idx
        plt.errorbar(xpos, step_value, step_err, c=color, alpha=1.)
        plt.plot(xpos, step_value, marker=marker, c=color, label=label)
        
    labels.append(prop_label)

plt.xticks(np.arange(len(labels)), labels=labels)
plt.axhline(0., c='k')
plt.xlim(-0.5, len(labels) - 0.5)
plt.ylim(-0.02, 0.17)

plt.ylabel('Step size (mag)')
plt.legend()
plt.tight_layout()

plt.savefig('/home/kyle/Desktop/unblinded_host_steps_peculiars_removed.png')
# plt.savefig('./figures/host_correlations_summary.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-28-de64287b860d>:29: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
  plt.tight_layout()


## Summary table

In [29]:
# All of these functions are defined in the Host galaxy correlations section. Make
# sure to run that first.
def print_row(f, mags, cut, label, cut_label):
    step_vars = [
        ('lssfr', '$\\Delta_{lsSFR}$', +1),
        ('gmass', '$\\Delta_{mass}$', -1),
    ]
    stat_str = "%20s & %20s" % (label, cut_label)
    for i, (var_name, var_label, var_sign) in enumerate(step_vars):
        var, var_err = analyze_host_variable(var_name, mags, cut, plot=False)
        stat_str += " & %.3f $\pm$ %.3f" % (var_sign * var, var_err)
        
    stat_str += " \\\\"
    latex_print(f, stat_str)
    
with open('./latex/host_steps.tex', 'w') as f:
    print_row(f, a.salt_hr, a.good_salt_mask, 'SALT2 + $x_1$', 'SALT2')
    print_row(f, a.salt_hr, a.good_salt_mask & a.good_mag_mask, 'SALT2 + $x_1$', 'SALT2 + max + train')
    print_row(f, rbtl_isomap_mags, a.good_salt_mask & a.good_mag_mask, 'RBTL + Isomap', 'SALT2 + max + train')
    print_row(f, salt_isomap_mags, a.good_salt_mask & a.good_mag_mask, 'SALT2 + Isomap', 'SALT2 + max + train')

AttributeError: 'ManifoldTwinsAnalysis' object has no attribute 'salt_hr'

# Attrition and LaTeX variables for things

## General variables

In [ ]:
# Define a bunch of functions to make things easier.
def latex_host_step(file, name, var, mags, mask):
    step, step_err = analyze_host_variable(var, mags, mask, plot=False)
    latex_command(file, name, '%.3f $\\pm$ %.3f', (np.abs(step), step_err))

a.fit_gp(kind='salt_raw', verbose=False)
salt_isomap_mags = a.corr_mags.copy()

a.fit_gp(verbose=False)
rbtl_isomap_mags = a.corr_mags.copy()

with open('latex/commands.tex', 'w') as f:
    latex_print(f, "")
    latex_command(f, 'numdatasetsne', '%d', len(a.dataset.targets))
    latex_command(f, 'numdatasetspectra', '%d', np.sum([len(i.spectra) for i in a.dataset.targets]))
    latex_print(f, "")
    latex_command(f, 'nummanifoldsne', '%d', len(a.targets))
    latex_command(f, 'nummanifoldspectra', '%d', len(a.spectra))
    latex_command(f, 'numinterpsne', '%d', np.sum(a.interp_mask))
    latex_print(f, "")
    latex_command(f, 'numsnftrain', '%d', np.sum([i.subset == 'training' for i in a.targets[a.interp_mask]]))
    latex_command(f, 'numsnfvalid', '%d', np.sum([i.subset == 'validation' for i in a.targets[a.interp_mask]]))
    latex_command(f, 'numsnfother', '%d', np.sum([i.subset not in ['training', 'validation'] for i in a.targets[a.interp_mask]]))
    latex_print(f, "")
    latex_command(f, 'numsnredshift', '%d', np.sum(a.interp_mask & (a.redshift_errs >= 0.004)))
    latex_command(f, 'numlowredshift', '%d', np.sum(a.interp_mask & (a.redshifts <= 0.02)))
    latex_command(f, 'numhighav', '%d', np.sum(a.interp_mask & (a.colors - np.nanmedian(a.colors) >= 0.5)))
    latex_print(f, "")
    latex_command(f, 'nummagsne', '%d', np.sum(a.interp_mask & a.redshift_color_mask))
    latex_command(f, 'nummagsnetrain', '%d', np.sum(a.good_mag_mask))
    latex_command(f, 'nummagsnevalidation', '%d', np.sum(a.interp_mask & a.redshift_color_mask & ~a.good_mag_mask))
    latex_print(f, "")
    latex_command(f, 'saltparammb', '%.2f', a.salt_MB)
    latex_command(f, 'saltparamalpha', '%.3f', a.salt_alpha)
    latex_command(f, 'saltparambeta', '%.2f', a.salt_beta)
    latex_command(f, 'saltparamsigmaint', '%.3f', a.salt_intrinsic_dispersion)
    # latex_command(f, 'saltparamrms', '%.3f', np.std(a.salt_hr[a.good_salt_mask]))
    latex_std(f, 'saltparamrms', a.salt_hr[a.good_salt_mask])
    latex_nmad(f, 'saltparamnmad', a.salt_hr[a.good_salt_mask])
    latex_command(f, 'saltparamwrms', '%.3f', a.salt_wrms)
    latex_command(f, 'saltparammindisp', '%.2f', np.min(a.salt_hr_uncertainties[a.good_salt_mask]))
    latex_command(f, 'saltparammaxdisp', '%.2f', np.max(a.salt_hr_uncertainties[a.good_salt_mask]))
    latex_print(f, "")
    latex_std(f, 'rawrbtlmagstd', a.mags[a.good_mag_mask])
    latex_nmad(f, 'rawrbtlmagnmad', a.mags[a.good_mag_mask])
    # latex_print(f, "")
    # latex_command(f, 'twinrbtlmagstd', '%.3f', a.twins_rms)
    # latex_command(f, 'twinrbtlmagnmad', '%.3f', a.twins_nmad)
    latex_print(f, "")
    latex_std(f, 'saltcomprawrbtlmagstd', a.mags[a.good_mag_mask & a.good_salt_mask])
    latex_std(f, 'saltcompsaltmagstd', a.salt_hr[a.good_mag_mask & a.good_salt_mask])

    a.fit_gp(verbose=False, kind='salt_raw')
    gp_uncertainties = np.sqrt(np.diag(a.gp_hyperparameter_covariance))
    latex_print(f, "")
    latex_command(f, 'saltgpcolor', '%.2f $\\pm$ %.2f', (a.gp_hyperparameters[0], gp_uncertainties[0]))
    latex_command(f, 'saltgpintdisp', '%.3f $\\pm$ %.3f', (a.gp_hyperparameters[1], gp_uncertainties[1]))
    latex_command(f, 'saltgpkernelamp', '%.3f $\\pm$ %.3f', (np.abs(a.gp_hyperparameters[2]), gp_uncertainties[2]))
    latex_command(f, 'saltgpkernellengthscale', '%.2f $\\pm$ %.2f', (a.gp_hyperparameters[3], gp_uncertainties[3]))
    latex_std(f, 'saltgprms', a.corr_mags[a.good_salt_mask & a.interp_mask])
    latex_std(f, 'saltgpcompsaltrms', a.salt_hr[a.good_salt_mask & a.interp_mask])

    a.fit_gp(verbose=False)
    gp_uncertainties = np.sqrt(np.diag(a.gp_hyperparameter_covariance))
    latex_print(f, "")
    latex_command(f, 'rbtlgpcolor', '%.2f $\\pm$ %.2f', (a.fiducial_rv * (1 + a.gp_hyperparameters[0]), a.fiducial_rv * gp_uncertainties[0]))
    latex_command(f, 'rbtlgpintdisp', '%.3f $\\pm$ %.3f', (a.gp_hyperparameters[1], gp_uncertainties[1]))
    latex_command(f, 'rbtlgpkernelamp', '%.3f $\\pm$ %.3f', (np.abs(a.gp_hyperparameters[2]), gp_uncertainties[2]))
    latex_command(f, 'rbtlgpkernellengthscale', '%.2f $\\pm$ %.2f', (a.gp_hyperparameters[3], gp_uncertainties[3]))

    latex_print(f, "")
    latex_std(f, 'rbtlgprms', a.corr_mags[a.good_mag_mask])
    latex_command(f, 'rbtlgpnmad', '%.3f', math.nmad(a.corr_mags[a.good_mag_mask]))

    latex_print(f, "")
    x1 = a.salt_hr[(a.embedding[:, 0] < 3) & a.good_salt_mask & a.interp_mask]
    x2 = a.salt_hr[(a.embedding[:, 0] > 3) & a.good_salt_mask & a.interp_mask]
    m1 = np.mean(x1)
    m2 = np.mean(x2)
    err1 = np.std(x1) / np.sqrt(len(x1))
    err2 = np.std(x2) / np.sqrt(len(x2))
    latex_command(f, 'saltisomapdiff', '%.3f $\\pm$ %.3f', (np.abs(m1-m2), np.sqrt(err1**2 + err2**2)))

    latex_print(f, "")
    latex_command(f, 'pecvelcontribution', '%.3f', np.sqrt(np.mean(a.get_peculiar_velocity_uncertainty()[a.good_mag_mask & a.good_salt_mask]**2)))

    latex_print(f, "")
    latex_host_step(f, 'lssfrsaltxifull', 'lssfr', a.salt_hr, a.good_salt_mask)
    latex_host_step(f, 'gmasssaltxifull', 'gmass', a.salt_hr, a.good_salt_mask)
    latex_host_step(f, 'lssfrsaltxicut', 'lssfr', a.salt_hr, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'gmasssaltxicut', 'gmass', a.salt_hr, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'lssfrsaltisomapcut', 'lssfr', salt_isomap_mags, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'gmasssaltisomapcut', 'gmass', salt_isomap_mags, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'lssfrrbtlisomapcut', 'lssfr', a.corr_mags, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'gmassrbtlisomapcut', 'gmass', a.corr_mags, a.good_salt_mask & a.good_mag_mask)
    latex_command(f, 'hostcutsnsnetrain', '%d', np.sum(a.good_mag_mask & a.good_salt_mask & a.host_mask))
    latex_command(f, 'hostcutsnsnefull', '%d', np.sum(a.redshift_color_mask & a.interp_mask & a.good_salt_mask & a.host_mask))

## Standardization comparison

In [20]:
def int_disp(vals, pec_vel_disps, axis=None):
    std = np.std(vals, ddof=1, axis=axis)
    corr = np.mean(pec_vel_disps**2, axis=axis)
    corr_std = np.sqrt(np.clip(std**2 - corr, 0, None))

    return corr_std

def get_stat_str(all_mags, cut, function, *args):
    line_str = ""
    for mags in all_mags:
        if line_str:
            line_str += " &"
            
        use_mags = mags[cut]
        if np.any(np.isnan(use_mags)):
            line_str += "%20s" % "--"
        else:
            res, res_err = math.bootstrap_statistic(function, use_mags, *args)
            line_str += "%7.3f $\pm$ %6.3f" % (res, res_err)
    
    line_str += " \\\\"
    
    return line_str

def make_table(f, all_mags, cut, label):
    stats = {
        'NMAD': (math.nmad,),
        'Standard deviation': (np.std,),
        'Pec. vel. removed': (int_disp, a.get_peculiar_velocity_uncertainty()[cut]),
    }
    for i, (stat_name, stat_args) in enumerate(stats.items()):
        if len(label) > i:
            prefix = label[i]
        else:
            prefix = ""
            
        if i == 0:
            num_sne = "%d" % np.sum(cut)
        else:
            num_sne = ""
            
        stat_str = get_stat_str(all_mags, cut, *stat_args)
        utils.latex_print(f, "%20s & %5s & %20s & %s" % (prefix, num_sne, stat_name, stat_str))
        
good_mag_mask = a.uncertainty_mask & a.redshift_color_mask
good_salt_mask = a.redshift_color_mask & a.salt_mask
good_salt_isomap_mask = a.uncertainty_mask & a.redshift_color_mask & a.salt_mask

# RBTL only
rbtl_mags = a.rbtl_mags.copy()
rbtl_mags[~good_mag_mask] = np.nan
        
# RBTL + Isomap
a.fit_gp()
rbtl_isomap_mags = a.corr_mags.copy()
rbtl_isomap_mags[~good_mag_mask] = np.nan

# SALT2
salt_mags = a.salt_hr.copy()
salt_mags[~good_salt_mask] = np.nan

# SALT2 + Isomap
a.fit_gp(kind='salt_raw')
salt_isomap_mags = a.corr_mags.copy()
salt_isomap_mags[~good_salt_isomap_mask] = np.nan

all_mags = [rbtl_mags, rbtl_isomap_mags, salt_mags, salt_isomap_mags]

with open('./latex/dispersions.tex', 'w') as f:
    # RBTL only
    make_table(f, all_mags, good_mag_mask, ['Maximum spectrum', '+ training cuts'])
    utils.latex_print(f, "\hline")

    # SALT2 only
    make_table(f, all_mags, good_salt_mask, ['SALT2 fit cuts'])
    utils.latex_print(f, "\hline")

    # SALT2 + Isomap
    make_table(f, all_mags, good_salt_isomap_mask, ['Maximum spectrum', '+ SALT2 fit cuts'])
    utils.latex_print(f, "\hline")

    # All
    make_table(f, all_mags, good_salt_isomap_mask, ['All cuts'])

Fitting GP hyperparameters...
    Fit result:           Optimization terminated successfully.
    Color scale:          -0.127 ± 0.051
    Intrinsic dispersion: 0.064 ± 0.009 mag
    GP kernel amplitude:  0.177 ± 0.089 mag
    GP length scale:      5.553 ± 3.271
    Fit NMAD:             0.081 mag
    Fit std:              0.100 mag
Fitting GP hyperparameters...
    Fit result:           Desired error not necessarily achieved due to precision loss.
    Color scale:          2.799 ± 0.151
    Intrinsic dispersion: -0.087 ± 0.009 mag
    GP kernel amplitude:  0.424 ± 0.250 mag
    GP length scale:      9.365 ± 5.524
    Fit NMAD:             0.103 mag
    Fit std:              0.117 mag
    Maximum spectrum &   134 &                 NMAD &   0.108 $\pm$  0.013 &  0.081 $\pm$  0.011 &                  -- &                  -- \\
     + training cuts &       &   Standard deviation &   0.130 $\pm$  0.010 &  0.100 $\pm$  0.008 &                  -- &                  -- \\
                  